In [1]:
#data challenge development

### Importing Library

In [2]:
import pandas as pd
import numpy as np

### Import the Data

In [3]:
#read employees data
dtypes_employees = {'employe_id': int, 'branch_id': int, 'salary': int} #, 'join_date': str, 'resign_date': str}
#parse_dates_employees = ['join_date', 'resign_date']
employees = pd.read_csv('employees.csv', dtype=dtypes_employees, usecols=['employe_id', 'branch_id', 'salary']) #parse_dates=parse_dates_employees,

#read timesheets data
dtypes_timesheets = {'timesheet_id': int, 'employee_id': int, 'date': str, 'checkin': str, 'checkout': str}
parse_dates_timesheets = ['date', 'checkin', 'checkout']
timesheets = pd.read_csv('timesheets.csv', dtype=dtypes_timesheets, parse_dates=parse_dates_timesheets)

In [4]:
#print(employees.info())
#print(timesheets.info())
#employees.describe()
#timesheets.describe()

#### preprocessing data

In [5]:
#check duplicated
employees.duplicated(subset=['employe_id']).any()

True

In [6]:
#see the list of duplicated data
employees[employees.duplicated(subset=['employe_id'], keep=False)]

,employe_id,branch_id,salary
163,218078,1,10500000
164,218078,1,13000000


In [7]:
#remove duplicate and keep the latest one
employees = employees.drop_duplicates(subset=['employe_id'], keep='last')

In [131]:
#handling missing value on checkout fields, approach is fill with the average time
timesheets['checkout'].fillna(timesheets.checkout.mean(), inplace=True)

In [132]:
#create new feature, working_hours as checkin and checkout difference
timesheets['working_hours'] = (timesheets.checkout - timesheets.checkin).astype('timedelta64[h]')

In [133]:
#merge employee and timesheet data
merge_df = employees.merge(timesheets, left_on=['employe_id'], right_on=['employee_id'], how='left')

In [9]:
merge_df

,employe_id,branch_id,salary,timesheet_id,employee_id,date,checkin,checkout
0,1,3,7500000,25475424.0,1.0,2019-10-03,2022-06-15 09:02:00,2022-06-15 19:19:00
1,1,3,7500000,25565087.0,1.0,2019-10-04,2022-06-15 08:42:00,2022-06-15 17:48:00
2,1,3,7500000,25617630.0,1.0,2019-10-07,2022-06-15 08:56:01,NaT
3,1,3,7500000,25722471.0,1.0,2019-10-08,2022-06-15 08:34:53,2022-06-15 17:57:57
4,1,3,7500000,25782958.0,1.0,2019-10-09,2022-06-15 08:32:47,2022-06-15 19:00:00
...,...,...,...,...,...,...,...,...
39715,340184,1,11500000,69972572.0,340184.0,2020-12-18,2022-06-15 08:00:00,2022-06-15 16:00:00
39716,340184,1,11500000,70041570.0,340184.0,2020-12-19,2022-06-15 07:51:03,2022-06-15 16:04:12
39717,340184,1,11500000,70049798.0,340184.0,2020-12-20,2022-06-15 07:51:45,2022-06-15 16:03:28
39718,340184,1,11500000,70434887.0,340184.0,2020-12-21,2022-06-15 19:56:50,NaT


In [37]:
timesheets[timesheets.duplicated(subset=['employee_id'], keep=False)].sort_values(by='employee_id')

,timesheet_id,employee_id,date,checkin,checkout
20703,45297206,1,2020-05-14,11:03:01,18:17:32
2793,26989264,1,2019-10-24,08:52:22,18:33:38
16213,41091166,1,2020-03-31,10:58:43,17:48:46
22996,46879009,1,2020-06-03,10:28:28,18:07:14
5459,29163014,1,2019-11-22,09:07:17,18:04:13
...,...,...,...,...,...
39685,70434887,340184,2020-12-21,19:56:50,NaN
39633,70049798,340184,2020-12-20,07:51:45,16:03:28
39697,70481740,340184,2020-12-22,15:45:27,00:05:46
39527,70041570,340184,2020-12-19,07:51:03,16:04:12


In [36]:
timesheets.duplicated(subset=['employee_id']).any()

True

In [ ]:
df['yyyy'] = pd.to_datetime(df['StartDate']).dt.year
df['mm'] = pd.to_datetime(df['StartDate']).dt.month

In [11]:
timesheets.columns

Index(['timesheet_id', 'employee_id', 'date', 'checkin', 'checkout'], dtype='object')

In [136]:
def get_salary_per_hour(df):
    cols = ['employe_id', 'branch_id', 'salary', 'timesheet_id', 'employee_id', 'date', 'checkin', 'checkout', 'working_hours']
    return (df.groupby(['date','branch_id'], as_index=False)
                .apply(lambda x: pd.DataFrame({
                    'year'              : x['date'].dt.year.astype(str),
                    'month'             : x['date'].dt.month.astype(str),
                    'branch_id'         : x['branch_id'],
                    'salary_per_hour'   : x['salary']/((x['checkout'] - x['checkin']).astype('timedelta64[h]'))
                }))
                )

In [145]:
def get_salary_per_hour(df):
    cols = ['employe_id', 'branch_id', 'salary', 'timesheet_id', 'employee_id', 'date', 'checkin', 'checkout', 'working_hours']
    return ((df.groupby(['date','branch_id'], as_index=False)[["salary", "working_hours"]].sum())
                .apply(lambda x: pd.DataFrame({
                    'year'              : x['date'].dt.year.astype(str),
                    'month'             : x['date'].dt.month.astype(str),
                    'branch_id'         : x['branch_id'],
                    'salary_per_hour'   : x['salary']/x['working_hours']
                }))
                )

In [ ]:
(x['salary'],x['checkout'].sum)

In [146]:
asdads = merge_df.groupby(['date','branch_id'], as_index=False)[["salary", "working_hours"]].sum()
asdads

,date,branch_id,salary,working_hours
0,2019-08-21,1,21500000,17.0
1,2019-08-21,2,36000000,23.0
2,2019-08-22,1,13500000,9.0
3,2019-08-22,2,22500000,8.0
4,2019-08-23,1,8000000,9.0
...,...,...,...,...
6051,2020-12-23,2631,9000000,10.0
6052,2020-12-23,2633,21500000,20.0
6053,2020-12-23,3092,29500000,16.0
6054,2020-12-23,11265,11000000,9.0


In [147]:
asd = get_salary_per_hour(merge_df)
asd

KeyError: 'date'

In [122]:
tes = merge_df.copy()
tes['checkout'].fillna((tes.checkout.mean()), inplace=True)
tes['working_hours'] = (tes.checkout - tes.checkin).astype('timedelta64[h]')
tes

,employe_id,branch_id,salary,timesheet_id,employee_id,date,checkin,checkout,working_hours
0,1,3,7500000,25475424.0,1.0,2019-10-03,2022-06-15 09:02:00,2022-06-15 19:19:00.000000000,10.0
1,1,3,7500000,25565087.0,1.0,2019-10-04,2022-06-15 08:42:00,2022-06-15 17:48:00.000000000,9.0
2,1,3,7500000,25617630.0,1.0,2019-10-07,2022-06-15 08:56:01,2022-06-15 18:07:30.866002944,9.0
3,1,3,7500000,25722471.0,1.0,2019-10-08,2022-06-15 08:34:53,2022-06-15 17:57:57.000000000,9.0
4,1,3,7500000,25782958.0,1.0,2019-10-09,2022-06-15 08:32:47,2022-06-15 19:00:00.000000000,10.0
...,...,...,...,...,...,...,...,...,...
39715,340184,1,11500000,69972572.0,340184.0,2020-12-18,2022-06-15 08:00:00,2022-06-15 16:00:00.000000000,8.0
39716,340184,1,11500000,70041570.0,340184.0,2020-12-19,2022-06-15 07:51:03,2022-06-15 16:04:12.000000000,8.0
39717,340184,1,11500000,70049798.0,340184.0,2020-12-20,2022-06-15 07:51:45,2022-06-15 16:03:28.000000000,8.0
39718,340184,1,11500000,70434887.0,340184.0,2020-12-21,2022-06-15 19:56:50,2022-06-15 18:07:30.866002944,-2.0


In [128]:
tes2 = tes.groupby(['date','branch_id'], as_index=False)[["salary", "working_hours"]].sum()
tes2

,date,branch_id,salary,working_hours
0,2019-08-21,1,21500000,17.0
1,2019-08-21,2,36000000,23.0
2,2019-08-22,1,13500000,9.0
3,2019-08-22,2,22500000,8.0
4,2019-08-23,1,8000000,9.0
...,...,...,...,...
6051,2020-12-23,2631,9000000,10.0
6052,2020-12-23,2633,21500000,20.0
6053,2020-12-23,3092,29500000,16.0
6054,2020-12-23,11265,11000000,9.0


In [119]:
tes.first()

salary  timesheet_id  employee_id  \
date       branch_id employe_id                                        
2019-08-21 1         21          13500000    23907434.0         21.0   
                     22           8000000    23907433.0         22.0   
           2         60          12500000    23907437.0         60.0   
                     63          13500000    23907435.0         63.0   
                     66          10000000    23907432.0         66.0   
...                                   ...           ...          ...   
2020-12-23 3092      21798        7500000    66950888.0      21798.0   
                     26631        9000000    66950966.0      26631.0   
                     33936        7500000    66951043.0      33936.0   
           11265     251119      11000000    70641497.0     251119.0   
2020-12-30 1         15           6000000    70047548.0         15.0   

                                            checkin            checkout  
date       branch_id employe_id                                          
2019-08-21 1         21         2022-06-15 09:45:08 2022-06-15 18:24:06  
                     22         2022-06-15 08:56:34 2022-06-15 18:00:48  
           2         60         2022-06-15 09:56:05 2022-06-15 17:31:08  
                     63         2022-06-15 09:55:47                 NaT  
                     66         2022-06-15 08:13:31 2022-06-15 17:05:02  
...                                             ...                 ...  
2020-12-23 3092      21798      2022-06-15 17:08:33                 NaT  
                     26631      2022-06-15 14:10:29                 NaT  
                     33936      2022-06-15 13:56:04                 NaT  
           11265     251119     2022-06-15 08:51:12                 NaT  
2020-12-30 1         15         2022-06-15 09:00:00 2022-06-15 18:53:00  

[39710 rows x 5 columns]

In [97]:
tes = (timesheets.checkout - timesheets.checkin).astype('timedelta64[h]')
tes

0        8.0
1        9.0
2        8.0
3        NaN
4        7.0
        ... 
39709    8.0
39710    9.0
39711    NaN
39712    9.0
39713    NaN
Length: 39714, dtype: float64

In [106]:
tes

,timesheet_id,employee_id,date,checkin,checkout
0,23907432,66,2019-08-21,2022-06-15 08:13:31,2022-06-15 17:05:02
1,23907433,22,2019-08-21,2022-06-15 08:56:34,2022-06-15 18:00:48
2,23907434,21,2019-08-21,2022-06-15 09:45:08,2022-06-15 18:24:06
3,23907435,63,2019-08-21,2022-06-15 09:55:47,NaT
4,23907437,60,2019-08-21,2022-06-15 09:56:05,2022-06-15 17:31:08
...,...,...,...,...,...
39709,70649593,59,2020-12-23,2022-06-15 08:05:29,2022-06-15 17:05:18
39710,70649597,109100,2020-12-23,2022-06-15 08:05:30,2022-06-15 17:05:30
39711,70650186,9908,2020-12-23,2022-06-15 08:06:52,NaT
39712,70650458,70,2020-12-23,2022-06-15 08:07:33,2022-06-15 17:13:27


In [108]:
timesheets.checkout.mean()

Timestamp('2022-06-15 18:07:30.866002688')